# Lab 2.2 – RAG with Watsonx, Elasticsearch, and LangChain (Google Colab Version)

![watsonx](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)

---

## 🚀 Run this notebook in Google Colab

**Prerequisites:**
- IBM Cloud API Key ([Create one here](https://cloud.ibm.com/iam/apikeys))
- Watsonx Space ID ([Create a deployment space](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=wx))
- **Elasticsearch Cloud Endpoint** (see setup instructions below)

⚠️ **Important:** This notebook requires an **Elasticsearch Cloud** instance. You can:
- Use [Elastic Cloud](https://cloud.elastic.co/) (14-day free trial available)
- Use [IBM Cloud Databases for Elasticsearch](https://cloud.ibm.com/catalog/services/databases-for-elasticsearch)

This notebook demonstrates **Retrieval Augmented Generation (RAG)** using:
- **Watsonx.ai** for LLM inference
- **Elasticsearch** as vector database
- **LangChain** for RAG orchestration

---

## Step 1: Install Dependencies

In [ ]:
# Install all required dependencies
!pip install -qU wget
!pip install -qU pandas
!pip install -qU humanize
!pip install -qU "langchain>=0.3,<0.4"
!pip install -qU "ibm_watsonx_ai>=1.1.22"
!pip install -qU "langchain_ibm>=0.3,<0.4"
!pip install -qU "langchain-huggingface>=0.1,<0.2"
!pip install -qU "langchain-elasticsearch>=0.3,<0.4"
!pip install -qU sentence-transformers

print("✅ All dependencies installed successfully!")

## Step 2: Configure Watsonx Credentials

In [ ]:
import os
import getpass
import pandas as pd
import humanize
import random

# Prompt for Watsonx credentials
watsonx_api_key = getpass.getpass("Enter IBM Cloud API Key: ")
space_id = getpass.getpass("Enter Watsonx Space ID (not project ID): ")
url = "https://us-south.ml.cloud.ibm.com"

os.environ["WATSONX_APIKEY"] = watsonx_api_key
os.environ["SPACE_ID"] = space_id

print("✅ Watsonx credentials configured!")

In [ ]:
from ibm_watsonx_ai import Credentials, APIClient

credentials = Credentials(
    url=url,
    api_key=watsonx_api_key,
)

api_client = APIClient(credentials=credentials, space_id=space_id)

print("✅ Watsonx API Client initialized!")

## Step 3: Configure Elasticsearch Credentials

You'll need:
- Elasticsearch URL (e.g., `https://my-cluster.es.us-east-1.aws.found.io:9243`)
- Username (usually `elastic`)
- Password
- SSL Certificate (base64 encoded) - Optional for Elastic Cloud

**Note:** For Elastic Cloud, you can often skip the SSL certificate.

In [ ]:
elasticsearch_url = input("Enter Elasticsearch URL (e.g., https://xxx.es.cloud.es.io:9243): ")
elasticsearch_username = input("Enter Elasticsearch username (usually 'elastic'): ")
elasticsearch_password = getpass.getpass("Enter Elasticsearch password: ")

print("\n✅ Elasticsearch credentials configured!")
print("\nℹ️  For SSL certificate, you can leave it empty if using Elastic Cloud.")
ssl_certificate_input = input("Enter SSL certificate (base64) or press Enter to skip: ")
ssl_certificate = ssl_certificate_input if ssl_certificate_input else None

## Step 4: Download Test Data

In [ ]:
import wget

questions_test_filename = 'questions_test.csv'
questions_train_filename = 'questions_train.csv'
questions_test_url = 'https://raw.github.com/IBM/watson-machine-learning-samples/master/cloud/data/RAG/questions_test.csv'
questions_train_url = 'https://raw.github.com/IBM/watson-machine-learning-samples/master/cloud/data/RAG/questions_train.csv'

if not os.path.isfile(questions_test_filename): 
    wget.download(questions_test_url, out=questions_test_filename)

if not os.path.isfile(questions_train_filename): 
    wget.download(questions_train_url, out=questions_train_filename)

test_data = pd.read_csv(questions_test_filename)
train_data = pd.read_csv(questions_train_filename)

print("\n✅ Test data downloaded!")
print(f"Train data shape: {train_data.shape}")
train_data.head()

## Step 5: Download Knowledge Base Documents

In [ ]:
knowledge_base_dir = "./knowledge_base"

os.makedirs(knowledge_base_dir, exist_ok=True)
   
documents_filename = 'knowledge_base/psgs.tsv'
documents_url = 'https://raw.github.com/IBM/watson-machine-learning-samples/master/cloud/data/RAG/psgs.tsv'

if not os.path.isfile(documents_filename): 
    wget.download(documents_url, out=documents_filename)

documents = pd.read_csv(f"{knowledge_base_dir}/psgs.tsv", sep='\t', header=0, nrows=1000)
documents['indextext'] = documents['title'].astype(str) + "\n" + documents['text']

print(f"\n✅ Loaded {len(documents)} documents for knowledge base")

## Step 6: Create LangChain Documents

In [ ]:
from langchain_core.documents import Document

lc_documents = [
    Document(page_content=text, metadata={"id": doc_id}) 
    for text, doc_id in zip(documents['indextext'], documents['id'])
]

print(f"✅ Created {len(lc_documents)} LangChain documents")

## Step 7: Create Embedding Function

We use HuggingFace's `all-MiniLM-L6-v2` model for embeddings.

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

emb_func = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

print("✅ Embedding function created!")

## Step 8: Initialize Watsonx LLM

In [ ]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from langchain_ibm import WatsonxLLM

text_model_id = api_client.foundation_models.TextModels.FLAN_UL2

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 50
}

watsonx_llm = WatsonxLLM(
    model_id=text_model_id,
    url=credentials.get("url"),
    apikey=credentials.get("apikey"),
    space_id=space_id,
    params=parameters
)

print("✅ Watsonx LLM initialized!")

## Step 9: Create Elasticsearch Connection

This creates a connection asset in Watsonx that can be reused.

In [ ]:
elasticsearch_data_source_type_id = api_client.connections.get_datasource_type_uid_by_name(
    "elasticsearch"
)

connection_properties = {
    "url": elasticsearch_url,
    "username": elasticsearch_username,
    "password": elasticsearch_password,
    "use_anonymous_access": "false",
}

if ssl_certificate:
    connection_properties["ssl_certificate"] = ssl_certificate

connections_details = api_client.connections.create(
    {
        api_client.connections.ConfigurationMetaNames.NAME: "Elasticsearch Connection (Colab)",
        api_client.connections.ConfigurationMetaNames.DESCRIPTION: "Connection created from Colab notebook",
        api_client.connections.ConfigurationMetaNames.DATASOURCE_TYPE: elasticsearch_data_source_type_id,
        api_client.connections.ConfigurationMetaNames.PROPERTIES: connection_properties,
    }
)

elasticsearch_connection_id = api_client.connections.get_id(connections_details)

print(f"✅ Elasticsearch connection created: {elasticsearch_connection_id}")

## Step 10: Create Elasticsearch Index and Load Documents

In [ ]:
from datetime import datetime
from ibm_watsonx_ai.foundation_models.extensions.rag.vector_stores.vector_store import VectorStore

index_name = f"elastic_index_{datetime.now().strftime('%Y_%m_%d_%H%M%S')}"

knowledge_base = VectorStore(
    api_client,
    connection_id=elasticsearch_connection_id,
    embeddings=emb_func,
    index_name=index_name,
)

elasticsearch_client = knowledge_base.get_client().client

print(f"✅ Elasticsearch index created: {index_name}")

## Step 11: Embed and Index Documents

⚠️ **This may take several minutes** to embed 1000 documents.

In [ ]:
print("Indexing documents... This may take a few minutes.")
stored_documents = knowledge_base.add_documents(content=lc_documents)

doc_count = knowledge_base.count()
print(f"\n✅ Indexed {doc_count} documents into Elasticsearch!")

## Step 12: Verify Index Structure

In [ ]:
# Show index structure
index_info = dict(elasticsearch_client.indices.get(index=index_name))
print("Index mapping:")
print(index_info[index_name]['mappings'])

## Step 13: Create RAG Chain

In [ ]:
from langchain.chains.retrieval_qa.base import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=watsonx_llm,
    chain_type="stuff",
    retriever=knowledge_base.as_langchain_retriever(),
    return_source_documents=True
)

print("✅ RAG chain created!")

## Step 14: Test RAG System

In [ ]:
questions_and_answers = {
    "names of founding fathers of the united states?": "Thomas Jefferson::James Madison::John Jay::George Washington::John Adams::Benjamin Franklin::Alexander Hamilton",
    "who played in the super bowl in 2013?": "Baltimore Ravens::San Francisco 49ers",
    "when did bucharest become the capital of romania?": "1862"
}

results = []

for question in questions_and_answers.keys():
    result = qa.invoke({"query": question})
    results.append(result)

print("✅ Generated answers for test questions!")

## Step 15: Display Results

In [ ]:
for idx, result in enumerate(results):
    print("=" * 80)
    print(f"Question: {result['query']}")
    print(f"Answer: {result['result']}")
    print(f"Expected: {questions_and_answers[result['query']]}")
    print("\nSource documents:")
    for i, doc in enumerate(result['source_documents'][:2], 1):
        print(f"  {i}. {doc.page_content[:200]}...")
    print()

## Step 16: Try Your Own Questions!

In [ ]:
your_question = input("Enter your question: ")
result = qa.invoke({"query": your_question})

print("\n" + "="*80)
print(f"Question: {result['query']}")
print("="*80)
print(f"Answer: {result['result']}")
print("\nRelevant sources:")
for i, doc in enumerate(result['source_documents'][:3], 1):
    print(f"\n{i}. {doc.page_content[:300]}...")

---

## Summary

You successfully completed this notebook! You learned how to:

✅ Set up Elasticsearch Cloud connection from Google Colab  
✅ Create embeddings using HuggingFace models  
✅ Index documents into Elasticsearch vector database  
✅ Implement RAG using LangChain with Watsonx and Elasticsearch  

**Next Steps:**
- Try different embedding models
- Experiment with different LLM models from Watsonx
- Index your own documents
- Deploy as an AI service (see original notebook for deployment code)

For more information:
- [Watsonx.ai Documentation](https://ibm.github.io/watsonx-ai-python-sdk/samples.html)
- [Elasticsearch Documentation](https://www.elastic.co/guide/index.html)

---

**Copyright © 2024 IBM. This notebook and its source code are released under the terms of the MIT License.**